In [1]:
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess

In [2]:
%run common.ipynb

In [3]:
train_df = pd.read_pickle("./train.pkl")
train_df.head()

,ari_time,ari_title,price
0,2006-10-20T20:16:16Z,"Inco's Net Soars on Higher Metal Prices, Break...",1
1,2006-10-23T11:51:36Z,EU Energy Chief Backs German Plan for Price Co...,1
2,2006-10-23T20:00:29Z,Ex-Plant Worker Shuster Pleads Guilty in Tradi...,1
3,2006-10-24T10:53:59Z,"Russia, Ukraine End Dispute That Cut Gas Supplies",1
4,2006-10-24T01:32:04Z,"Jim Cramer: Bare Escentuals, Allergan, Medicis...",1


In [4]:
train_reduce_df = pd.read_pickle("./train_reduce.pkl")
train_reduce_df.head()

,ari_time,ari_title,price
0,0.000000,,1
1,0.001181,,1
2,0.001574,,1
3,0.001968,,1
4,0.002755,balda say investor asks,0


In [5]:
%%time

corpus = list(map(simple_preprocess, train_reduce_df['ari_title']))
corpus += list(map(simple_preprocess, train_df['ari_title']))

print(corpus[0:10])

[[], [], [], [], ['balda', 'say', 'investor', 'asks'], [], ['ace', 'lift', 'pay'], [], [], []]
CPU times: user 1.73 s, sys: 23.7 ms, total: 1.75 s
Wall time: 1.75 s


In [6]:
%%time

model = Word2Vec(corpus, size=EMBEDDING_DIM, iter=10)
model.save('word2vec.model')

CPU times: user 16 s, sys: 124 ms, total: 16.1 s
Wall time: 6.05 s


In [7]:
# [Optional] Calculate max length among all titles for checking, ref MAX_WORD_COUNT in constants.ipynb
max_word_count = np.max(list(map(len, corpus)))
print("Max word count is %d among %d titles." % (max_word_count, len(train_df['ari_title'])))

Max word count is 579 among 54947 titles.


In [8]:
# [Optional] Calculate number of unseen words for checking
words = {}
word_count = 0
unseen_count = 0
for wl in corpus:
    for word in wl:
        word_count += 1
        if word in words:
            continue
        else:
            words[word] = True
        if word not in model.wv.vocab:
            unseen_count += 1            
non_repeated_words = len(words)

print(model)
print(train_df.shape)
print(word_count, non_repeated_words, unseen_count)
print(unseen_count / non_repeated_words)

Word2Vec(vocab=8723, size=50, alpha=0.025)
(54947, 3)
590964 28337 19614
0.6921692486854643
